In [5]:
import numpy as np
import nibabel as nib
import torchio as tio
import torch
from pathlib import Path

In [86]:
base_path = Path("..") / "data" / "segthor_train" / "train"

In [90]:
noise_transform = tio.transforms.RandomNoise(mean=0, std=0.05, p=0.5) # some higher std or p might be good
elastic_transform = tio.RandomElasticDeformation(
    num_control_points=7,  # More control points for finer, local deformations
    max_displacement=(3, 3, 3),  # Maximum displacement in mm, we still need to talk about this
    locked_borders=2  # Avoids border distortions
)
composed_transform = tio.transforms.Compose([
    elastic_transform,
    noise_transform
])

In [94]:
for patient_ID in range(1, 41):
    patient_ID = "{:02d}".format(patient_ID)
    print(f"working on patient", patient_ID)
    path_to_ground_truth = base_path / f"Patient_{patient_ID}" / "GT.nii" / "real_GT.nii.gz"
    path_to_patient = base_path / f"Patient_{patient_ID}" / f"Patient_{patient_ID}.nii" / f"Patient_{patient_ID}.nii"

    subject = tio.Subject(
        patient=tio.ScalarImage(path_to_patient),
        GT=tio.LabelMap(path_to_ground_truth)
    )
    transformed_subject = composed_transform(subject)

    augmented_patient_path = base_path / f"Patient_{patient_ID}" / "augmented" / "augmented_patient.nii"
    augmented_GT_path = base_path / f"Patient_{patient_ID}" / "augmented" / "augmented_GT.nii"
    
    # Create the directories if they don't exist
    augmented_patient_path.parent.mkdir(parents=True, exist_ok=True)
    augmented_GT_path.parent.mkdir(parents=True, exist_ok=True)
    
    # save the transformed images
    transformed_subject['patient'].save(augmented_patient_path)
    transformed_subject['GT'].save(augmented_GT_path)
print("done")